In [7]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [8]:
training_files = 1
data_files = glob.glob('bid_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))

In [9]:
train_data.head()

,bid_pos,go_alone,def_alone,def_pos_rel,turn_card_level,bid_turn_suit,bid_next_suit,bid_green_suit,top_trump_strg,top_2_trump_strg,top_3_trump_strg,num_trump,num_next,num_voids,num_singletons,num_off_aces,num_tricks,points
0,0,0,0,0,6,1,0,0,7,7,7,2,0,1,1,0,2,-2
1,1,1,0,0,6,1,0,0,0,0,0,0,1,1,1,2,3,1
2,2,1,0,0,6,1,0,0,0,0,0,0,2,1,1,1,0,-2
3,1,1,1,1,6,1,0,0,0,0,0,0,2,1,1,1,3,4
4,2,1,1,1,6,1,0,0,10,13,13,3,1,1,2,0,4,4


In [10]:
label = 'points'
train_data[label].describe()

count    150196.000000
mean          0.145756
std           2.624142
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           4.000000
max           4.000000
Name: points, dtype: float64

In [12]:
exclude = []
outputs = ['num_tricks', 'points']
outputs.remove(label)

type_map = {k: 'int' for k in list(train_data)}
type_map['bid_pos'] = 'category'
metadata = FeatureMetadata(type_map_raw=type_map)

predictor = TabularPredictor(problem_type='regression', label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), feature_metadata=metadata)

No path specified. Models will be saved in: "AutogluonModels/ag-20240904_042954"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       4.92 GB / 15.34 GB (32.1%)
Disk Space Avail:   126.55 GB / 452.95 GB (27.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_

[1000]	valid_set's rmse: 1.96807
[2000]	valid_set's rmse: 1.96601


	-1.965	 = Validation score   (-root_mean_squared_error)
	8.76s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-2.1184	 = Validation score   (-root_mean_squared_error)
	35.66s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	-1.9731	 = Validation score   (-root_mean_squared_error)
	72.19s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-2.0962	 = Validation score   (-root_mean_squared_error)
	28.45s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-1.9726	 = Validation score   (-root_mean_squared_error)
	140.57s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	-1.9698	 = Validation score   (-root_mean_squared_error)
	3.29s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-2.1873	 = Validation score   (-root_mean_squared_error)
	44.84s	 = Training   runtime
	0.01s	 = Validation

In [13]:
test_data = TabularDataset(data_files[training_files])

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: bid_data/bid_model_2-20240903_115017.tsv | Columns = 18 / 18 | Rows = 153008 -> 153008


0   -1.205142
1    0.676328
2   -1.288986
3   -1.398261
4    3.916028
Name: points, dtype: float32

In [14]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -1.9797934153214496,
 'mean_squared_error': -3.91958196735017,
 'mean_absolute_error': -1.4942253849881875,
 'r2': 0.43185309054303544,
 'pearsonr': 0.6572202002537143,
 'median_absolute_error': -1.1766999959945679}

In [15]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-1.978256,-1.973065,root_mean_squared_error,0.305183,0.008345,72.189497,0.305183,0.008345,72.189497,1,True,6
1,NeuralNetFastAI,-1.979406,-1.972594,root_mean_squared_error,1.635734,0.037182,140.573978,1.635734,0.037182,140.573978,1,True,8
2,LightGBMLarge,-1.979644,-1.969221,root_mean_squared_error,1.569494,0.032027,3.413145,1.569494,0.032027,3.413145,1,True,11
3,XGBoost,-1.979706,-1.969793,root_mean_squared_error,0.974257,0.025637,3.291310,0.974257,0.025637,3.291310,1,True,9
4,WeightedEnsemble_L2,-1.979793,-1.962429,root_mean_squared_error,6.284697,0.117661,149.360271,0.002620,0.000614,0.025761,2,True,12
5,LightGBMXT,-1.980814,-1.982222,root_mean_squared_error,1.745215,0.029922,2.931902,1.745215,0.029922,2.931902,1,True,3
6,LightGBM,-1.987395,-1.964999,root_mean_squared_error,4.646343,0.079865,8.760532,4.646343,0.079865,8.760532,1,True,4
7,ExtraTreesMSE,-2.109754,-2.096248,root_mean_squared_error,3.324547,0.088389,28.445657,3.324547,0.088389,28.445657,1,True,7
8,RandomForestMSE,-2.132998,-2.118357,root_mean_squared_error,3.307584,0.090647,35.660606,3.307584,0.090647,35.660606,1,True,5
9,NeuralNetTorch,-2.165332,-2.187315,root_mean_squared_error,0.280376,0.013016,44.839801,0.280376,0.013016,44.839801,1,True,10
